# 1. Connect to IB 

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

# 2. Market Segment Analysis

In [3]:
import market_analyzer as ma

# Initialize analyzer
analyzer = ma.IBMarketAnalyzer(ib)

# IMPORTANT: Set up the sectors first
analyzer.set_sectors()  # This was missing in the previous code

# Now get the full analysis
market_df, sector_df = analyzer.create_analysis_report()



Running market and sector analysis first...


In [4]:
sector_df.sort_values('Overall Score', inplace=True, ascending=False)
sector_df = sector_df.round(2)
sector_df

,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,4.96,5.69,-1.67,Increasing,Yes,Negative,1,1.0,1.0,9.00
1,Communication Services,XLC,1.11,1.85,-2.94,Increasing,Yes,Negative,2,2.0,3.0,7.67
2,Technology,XLK,1.01,1.74,-2.03,Increasing,Yes,Negative,3,3.0,2.0,7.33
3,Consumer_Staples,XLP,-2.32,-1.59,-2.96,Increasing,No,Negative,4,4.0,4.0,6.00
4,Healthcare,XLV,-4.27,-3.54,-4.35,Increasing,No,Negative,5,5.0,7.0,4.33
5,Financials,XLF,-4.38,-3.64,-3.75,Increasing,No,Negative,6,6.0,5.0,4.33
6,Utilities,XLU,-6.16,-5.43,-4.09,Increasing,No,Negative,7,7.0,6.0,3.33
7,Industrials,XLI,-6.19,-5.46,-4.80,Increasing,No,Negative,8,8.0,8.0,2.00
8,Real_Estate,XLRE,-7.48,-6.75,-6.81,Increasing,No,Negative,9,9.0,9.0,1.00
9,Materials,XLB,-9.04,-8.31,-6.89,Increasing,No,Negative,10,10.0,10.0,0.00


In [5]:


# Then filter the sectors in various ways:

custom_filter_df = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=False,
    min_momentum=-3,
    sort_by='Overall Score',
    min_score=5
)

custom_filter_list = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=True,
    min_momentum=-3,
    sort_by='Overall Score',
    min_score=5
)

# ib.disconnect()

print("=== Market Analysis ===")
display(market_df)

print("\n=== Sector Analysis ===")
display(sector_df)


try:


    print("\n=== Custom Filter DF ===")
    display(custom_filter_df)

    print("\n=== Custom Filter Sector List ===")
    display(custom_filter_list)

except Exception as e:
    print(f"An error occurred: {str(e)}")

status_dict = {
    "sector_analysis": len(sector_df),
    "stocks_scanned": 0,
    "stocks_validated": 0,
    "stocks_scored": 0
}

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings= None,
    activity_df = None,
    sector_etf_df = sector_df.round(2),
    fundamentals_df = None,
    ta_df = None,
    daily_ta_df = None
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

=== Market Analysis ===


,Metric,Description,Value,Interpretation
0,SPY Current Price,Current market price of SPY ETF,591.15,Current market price point
1,SPY 20-Day MA,20-day simple moving average - short-term trend indicator,601.14,Price above 20MA is bullish short-term
2,SPY 50-Day MA,50-day simple moving average - medium-term trend indicator,591.34,Price above 50MA is bullish medium-term
3,SPY 200-Day MA,200-day simple moving average - long-term trend indicator,551.29,Price above 200MA is bullish long-term
4,SPY VWAP,Volume Weighted Average Price - shows average price based on volume,528.64,Price above VWAP indicates buying pressure
5,Price vs 20MA (%),Percentage difference between current price and 20-day MA,-1.66,Positive % indicates price strength vs 20MA
6,Price vs 50MA (%),Percentage difference between current price and 50-day MA,-0.03,Positive % indicates price strength vs 50MA
7,Price vs 200MA (%),Percentage difference between current price and 200-day MA,7.23,Positive % indicates price strength vs 200MA
8,Price vs VWAP (%),Percentage difference between current price and VWAP,11.82,Positive % indicates current buying pressure
9,SPY RSI,Relative Strength Index (14-day) - momentum indicator (0-100),36.84,"RSI > 70 overbought, < 30 oversold"



=== Sector Analysis ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,4.96,5.69,-1.67,Increasing,Yes,Negative,1,1.0,1.0,9.00
1,Communication Services,XLC,1.11,1.85,-2.94,Increasing,Yes,Negative,2,2.0,3.0,7.67
2,Technology,XLK,1.01,1.74,-2.03,Increasing,Yes,Negative,3,3.0,2.0,7.33
3,Consumer_Staples,XLP,-2.32,-1.59,-2.96,Increasing,No,Negative,4,4.0,4.0,6.00
4,Healthcare,XLV,-4.27,-3.54,-4.35,Increasing,No,Negative,5,5.0,7.0,4.33
5,Financials,XLF,-4.38,-3.64,-3.75,Increasing,No,Negative,6,6.0,5.0,4.33
6,Utilities,XLU,-6.16,-5.43,-4.09,Increasing,No,Negative,7,7.0,6.0,3.33
7,Industrials,XLI,-6.19,-5.46,-4.80,Increasing,No,Negative,8,8.0,8.0,2.00
8,Real_Estate,XLRE,-7.48,-6.75,-6.81,Increasing,No,Negative,9,9.0,9.0,1.00
9,Materials,XLB,-9.04,-8.31,-6.89,Increasing,No,Negative,10,10.0,10.0,0.00



=== Custom Filter DF ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,4.96,5.69,-1.67,Increasing,Yes,Negative,1,1.0,1.0,9.00
1,Communication Services,XLC,1.11,1.85,-2.94,Increasing,Yes,Negative,2,2.0,3.0,7.67
2,Technology,XLK,1.01,1.74,-2.03,Increasing,Yes,Negative,3,3.0,2.0,7.33
3,Consumer_Staples,XLP,-2.32,-1.59,-2.96,Increasing,No,Negative,4,4.0,4.0,6.00



=== Custom Filter Sector List ===


(['Consumer_Discretionary',
  'Communication Services',
  'Technology',
  'Consumer_Staples'],
 ['XLY', 'XLC', 'XLK', 'XLP'])

True

# 3. Scanner

In [6]:
import scanner
import schedule
from datetime import datetime

market = schedule.MarketSchedule(scan_time='09:00', sleep_time=600) 
sbscan = scanner.StockbotScanner(ib)

# Scan settings in dict so it can be logged and sent via email.
scan_settings = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "scan_code": "TOP_PERC_GAIN",
    "price_min": 1,
    "price_max": 100,
    "volume": 100_000,  # Will be formatted as "100K"
    "market_cap": 100_000_000,  # Will be formatted as "100M"
    "change_percent": 4,
    "total_scanned": 0
}

if market.wait_for_scan_time():
    sbscan.multiscan(
        scan_code=scan_settings['scan_code'], 
        price=(scan_settings['price_min'], scan_settings['price_max']),
        volume=scan_settings['volume'], 
        change_perc=4, 
        market_cap=scan_settings['market_cap']/1_000_000,
        limit_each_cap=20
    )

scan_settings['total_scanned'] = len(sbscan.scan_results_df)
scan_settings

Market is closed today (Weekend)
Next trading day is Monday, December 23, 2024


TypeError: object of type 'NoneType' has no len()

In [ ]:

sbscan.scan_results_df.head(10) # Display the first 10 rows

,Rank,Symbol,Market Cap Range
0,0,RZLV,"(100.0, 1000.0)"
16,0,COMM,"(1000.0, 10000.0)"
22,0,BIRK,"(10000.0, 100000.0)"
1,1,MAPS,"(100.0, 1000.0)"
17,1,QUBT,"(1000.0, 10000.0)"
2,2,ARQQ,"(100.0, 1000.0)"
18,2,OKLO,"(1000.0, 10000.0)"
3,3,CHPT,"(100.0, 1000.0)"
19,3,TUYA,"(1000.0, 10000.0)"
4,4,QSI,"(100.0, 1000.0)"


# 3.1 OFFLINE SCAN TESTING 

In [3]:
import pandas as pd

# Create the data as a list of dictionaries
dfx = pd.DataFrame([
    {'Rank': 0, 'Symbol': 'RZLV', 'Market Cap Range': '(100.0, 1000.0)'},
    {'Rank': 16, 'Symbol': 'COMM', 'Market Cap Range': '(1000.0, 10000.0)'},
    {'Rank': 22, 'Symbol': 'BIRK', 'Market Cap Range': '(10000.0, 100000.0)'},
    # {'Rank': 1, 'Symbol': 'MAPS', 'Market Cap Range': '(100.0, 1000.0)'},
    # {'Rank': 17, 'Symbol': 'QUBT', 'Market Cap Range': '(1000.0, 10000.0)'},
    # {'Rank': 2, 'Symbol': 'ARQQ', 'Market Cap Range': '(100.0, 1000.0)'},
    # {'Rank': 18, 'Symbol': 'OKLO', 'Market Cap Range': '(1000.0, 10000.0)'},
    # {'Rank': 3, 'Symbol': 'CHPT', 'Market Cap Range': '(100.0, 1000.0)'},
    # {'Rank': 19, 'Symbol': 'TUYA', 'Market Cap Range': '(1000.0, 10000.0)'},
    # {'Rank': 4, 'Symbol': 'QSI', 'Market Cap Range': '(100.0, 1000.0)'}
])

# offline scan object
import scanner
sbscan = scanner.StockbotScanner(ib)
sbscan.scan_results_df = dfx

sbscan.scan_results_df

,Rank,Symbol,Market Cap Range
0,0,RZLV,"(100.0, 1000.0)"
1,16,COMM,"(1000.0, 10000.0)"
2,22,BIRK,"(10000.0, 100000.0)"


In [4]:
import pandas as pd
import stock
from strategies import ta


# getting the stock data


def update_scan_results(ib, scan_results_df, allowed_etfs):
    for s in scan_results_df['Symbol']:
        sx = stock.StockXDaily(ib, s)
        sx.req_fundamentals(max_days_old=1)  # gets the fundamentals from the IB API
        sx.req_ohlcv()
        fund_results = sx.get_funadmentals_validation_results(allowed_etfs)
        ta_results   = sx.get_TA_validation_results()

        # Find the index of the row with the symbol
        index = scan_results_df[scan_results_df['Symbol'] == s].index[0]

        # Update the row with the validation results
        for key, value in fund_results.items():
            scan_results_df.loc[index, key] = value
        
        # Update the row with the validation results
        for key, value in ta_results.items():
            scan_results_df.loc[index, key] = value

    return scan_results_df


allowed_etfs = ['XLK', 'XLY', 'XLI']
update_scan_results(ib, sbscan.scan_results_df, allowed_etfs)

Using cached data for RZLV (age: 2024-12-22 10:00:08)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\RZLV_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\RZLV_1_day.csv
Using cached data for COMM (age: 2024-12-22 10:09:08)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\COMM_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\COMM_1_day.csv
Using cached data for BIRK (age: 2024-12-22 10:09:10)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\BIRK_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\BIRK_1_day.csv


,Rank,Symbol,Market Cap Range,Sector1 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,0,RZLV,"(100.0, 1000.0)",True,True,True,1.0,True,False,False,False,False,True,True,True,True,0.56
1,16,COMM,"(1000.0, 10000.0)",True,True,True,1.0,True,True,True,False,False,False,False,True,False,0.44
2,22,BIRK,"(10000.0, 100000.0)",True,True,True,1.0,True,True,True,False,True,False,False,True,True,0.67


# 4. Daily Stock Validation (Fundamentals & TA)

In [ ]:
# permisaable ETFs
allowed_etfs = custom_filter_list[1]
# allowed_etfs = ['XLY', 'XLC', 'XLK']
print(f"Allowed ETFs: {allowed_etfs}")

# see how the scans are validatd here: C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\stock.py --> get_funadmentals_validation_results(), get_TA_validation_results()
sbscan.update_scan_results(allowed_etfs, ib, limit=100)

status_dict = {
    "sector_analysis": len(sector_df),
    "stocks_scanned": len(sbscan.scan_results_df),
    "stocks_validated": len(sbscan.fund_results_df['Fundamentals Passed']) + len(sbscan.ta_results_df['TA Passed']),
    "stocks_scored": 0
}


Allowed ETFs: ['XLY', 'XLC', 'XLK', 'XLP', 'XLF']
Using cached data for RZLV (age: 2024-12-17 18:04:40)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\RZLV_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\RZLV_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\COMM_1_day.csv
File not found : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\COMM_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\COMM_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\BIRK_1_day.csv
File not found : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\BIRK_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\BIRK_1_day.csv
Using cached data for MAPS (age: 2024-12-17 19:55:34)
Loading data

In [ ]:

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings   = scan_settings,
    sector_etf_df   = sector_df.round(2),
    fundamentals_df = sbscan.fund_results_df.sort_values('Fundamentals Passed', ascending=False),
    ta_df           = sbscan.ta_results_df.sort_values('TA Passed', ascending=False),
    daily_ta_df = None,
    activity_df = None,
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

# 5. Daily Stock Scores (TA)

In [ ]:
# AI prompt for charts 

# I want you to assess this image and assess it for each of the following criteria and provide a score for each element and a average score.:
# 1. Has the gapped up? (yes is good but not too excessive max 20-30%)
# 2. Has it broken out of a range? (yes is good) # imaginge a line joining the high points that the price has broken out of
# 3. was the price previously consolidating? (yes is good) # the longer the period of consolidation the better. 1-2 years is ideal. 
# 4. How many significant high points/resistance levels has the price broken throuhg? (more is better) # 2-3 is very good
# 5. Does the price have room to move? (yes is good) # look at the next resistance level and see how much room there is to move up. 2R is good.

# 6. Final Watch List